*ToDo*

02) pred_window und obs_window ausprobieren.
03) output nochmal genau anschauen.
04) was ist maskiert anschauen - und wie genau?
05) VRAM wenig ausgelastet. Batch size mal mit 320 ausprobieren, aber auch durchziehen.
06) Task pred: scheduler einfügen.
07) Analyse each var (sparsity)
08) loss per var/quality of varis loss (is this already ablation to forecast only one var)
09) Include sepsis definition
10) 

In [ ]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random 

from tqdm import tqdm
tqdm.pandas()
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
# data settings
test_cond = 0
sepsis_check = 1

if test_cond == 1:
    data_path = '/home/mitarb/fracarolli/files/231113_STraTS_new/mimic_iii_preprocessed_new_z_err20092.pkl'
    sample_divisor = 100
    number_of_epochs = 20
else:
    data_path = '/home/mitarb/fracarolli/files/231113_STraTS_new/mimic_iii_preprocessed_new_z_err20092.pkl'
    sample_divisor = 1
    number_of_epochs = 1000

In [ ]:
def set_seed(seed: int) -> None:
    """
    Set the random seed for modules torch, numpy and random.

    :param seed: random seed
    """
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available() and torch.cuda.device_count() > 0:
        torch.backends.cudnn.deterministic = True
        torch.cuda.manual_seed_all(seed)
#set_seed(42)

## Load forecast dataset into matrices.

In [ ]:
def inv_list(l, start=0):  # Create vind
    d = {}
    for i in range(len(l)):
        d[l[i]] = i+start
    return d


def f(x):
    mask   = [0 for i in range(V)]
    values = [0 for i in range(V)]
    for vv in x:  # tuple of ['vind','value']
        v = int(vv[0])-1  # shift index of vind
        mask[v] = 1
        values[v] = vv[1]  # get value
    return values+mask  # concat


def pad(x):
    if len(x) > 880:
        print(len(x))
    return x+[0]*(fore_max_len-len(x))

In [ ]:
"""

# Read data.
data, oc, train_ind, valid_ind, test_ind = pickle.load(open(data_path, 'rb'))
import itertools
if sepsis_check:
    test_ind = pd.read_csv('infec_test.csv', header=None)
    test_ind = list(itertools.chain(*test_ind.values.tolist()))
    test_ind = list(map(int, test_ind))

# Remove test patients.
print(len(train_ind))  # 28708
print(len(valid_ind))  # 7270
print(len(test_ind))  # 8880, 743 with sepsis_check
#raise Exception
data = data.merge(oc[['ts_ind', 'SUBJECT_ID']], on='ts_ind', how='left')
test_sub = oc.loc[oc.ts_ind.isin(test_ind)].SUBJECT_ID.unique()
data = data.loc[~data.SUBJECT_ID.isin(test_sub)]
oc = oc.loc[~oc.SUBJECT_ID.isin(test_sub)]
data.drop(columns=['SUBJECT_ID', 'TABLE'], inplace=True)
# Fix age.
data.loc[(data.variable == 'Age') & (data.value > 200), 'value'] = 91.4
# Get static data with mean fill and missingness indicator.
static_varis = ['Age', 'Gender']
ii = data.variable.isin(static_varis)
static_data = data.loc[ii]  # static data are 'Age' and 'Gender'
data = data.loc[~ii]  # ~ binary flip
# print('data\n',data)

static_var_to_ind = inv_list(static_varis)  # {'Age': 0, 'Gender': 1}
D = len(static_varis)  # 2 by definition
N = data.ts_ind.max()+1  # /= 12: 52861
print(N, D)
demo = np.zeros((int(N), int(D)))
for row in tqdm(static_data.itertuples()):
    print(row.ts_ind)
    demo[int(row.ts_ind), static_var_to_ind[row.variable]] = row.value
# print('Demo after tqdm command \n',demo[:10])
# Normalize static data.
means = demo.mean(axis=0, keepdims=True)  # quite sparse
stds = demo.std(axis=0, keepdims=True)
stds = (stds == 0)*1 + (stds != 0)*stds
demo = (demo-means)/stds
# print('Demo after normalisation \n',demo[:10])
# Get variable indices.
varis = sorted(list(set(data.variable)))
V = len(varis)  # 129 for \=12 with varis all variables except for static ones
print('varis', varis, V)
var_to_ind = inv_list(varis, start=1)
data['vind'] = data.variable.map(var_to_ind)
data = data[['ts_ind', 'vind', 'hour', 'value']].sort_values(by=['ts_ind', 'vind', 'hour'])
# print('data vind\n', data['vind'], '\n data\n',data)
# Find max_len.
fore_max_len = 880  # hard coded max_len of vars
# Get forecast inputs and outputs.
fore_times_ip = []
fore_values_ip = []
fore_varis_ip = []
fore_op = []
fore_op_awesome = []
fore_inds = []
for w in tqdm(range(25, 124, 4)):  # range(20, 124, 4), pred_window=2
    pred_data = data.loc[(data.hour>=w)&(data.hour<=w+24)]
    pred_data = pred_data.groupby(['ts_ind', 'vind']).agg({'value':'first'}).reset_index()
    pred_data['vind_value'] = pred_data[['vind', 'value']].values.tolist()
    pred_data = pred_data.groupby('ts_ind').agg({'vind_value':list}).reset_index()
    pred_data['vind_value'] = pred_data['vind_value'].apply(f)   

    obs_data = data.loc[(data.hour < w) & (data.hour >= w-24)]
    obs_data = obs_data.loc[obs_data.ts_ind.isin(pred_data.ts_ind)]
    obs_data = obs_data.groupby('ts_ind').head(fore_max_len)
    obs_data = obs_data.groupby('ts_ind').agg({'vind': list, 'hour': list, 'value': list}).reset_index()
    #Michi: Hier hole ich mir die 24 Stunden vor, und 24 Stunden nach einem bestimmten Zeitpunkt raus
    for pred_window  in range(-24, 24, 1):
        pred_data = data.loc[(data.hour >= w+pred_window) & (data.hour <= w+1 +pred_window)]
        pred_data = pred_data.groupby(['ts_ind', 'vind']).agg({'value': 'first'}).reset_index()
        pred_data['vind_value'+str(pred_window)] = pred_data[['vind', 'value']].values.tolist()
        pred_data = pred_data.groupby('ts_ind').agg({'vind_value'+str(pred_window): list}).reset_index()
        pred_data['vind_value'+str(pred_window)] = pred_data['vind_value'+str(pred_window)].apply(f)  # 721 entries with 2*129 vind_values
        obs_data = obs_data.merge(pred_data, on='ts_ind')

    for col in ['vind', 'hour', 'value']:
        obs_data[col] = obs_data[col].apply(pad)
    fore_op_awesome.append(np.array(list([list(obs_data['vind_value'+str(pred_window)]) for pred_window in range(-24, 24, 1)])))
    #print(fore_op_awesome[-1].shape)
    #raise Exception
    #fore_op.append(np.array(list(obs_data.vind_value)))
    fore_inds.append(np.array([int(x) for x in list(obs_data.ts_ind)]))
    fore_times_ip.append(np.array(list(obs_data.hour)))
    fore_values_ip.append(np.array(list(obs_data.value)))
    fore_varis_ip.append(np.array(list(obs_data.vind)))
    
del data
fore_times_ip = np.concatenate(fore_times_ip, axis=0)
fore_values_ip = np.concatenate(fore_values_ip, axis=0)
fore_varis_ip = np.concatenate(fore_varis_ip, axis=0)

fore_op_awesome = np.concatenate(fore_op_awesome, axis=1)
print(fore_op_awesome.shape)
fore_op = np.swapaxes(fore_op_awesome, 0, 1)
print(fore_op.shape)

#raise Exception
fore_inds = np.concatenate(fore_inds, axis=0)
fore_inds = fore_inds.astype(int)
fore_demo = demo[fore_inds]
# Get train and valid ts_ind for forecast task.
train_sub = oc.loc[oc.ts_ind.isin(train_ind)].SUBJECT_ID.unique()
valid_sub = oc.loc[oc.ts_ind.isin(valid_ind)].SUBJECT_ID.unique()
rem_sub = oc.loc[~oc.SUBJECT_ID.isin(np.concatenate((train_ind, valid_ind)))].SUBJECT_ID.unique()
bp = int(0.8*len(rem_sub))
train_sub = np.concatenate((train_sub, rem_sub[:bp]))
valid_sub = np.concatenate((valid_sub, rem_sub[bp:]))
train_ind = oc.loc[oc.SUBJECT_ID.isin(train_sub)].ts_ind.unique()  # Add remaining ts_ind s of train subjects.
valid_ind = oc.loc[oc.SUBJECT_ID.isin(valid_sub)].ts_ind.unique()  # Add remaining ts_ind s of train subjects.
# Generate 3 sets of inputs and outputs.
train_ind = np.argwhere(np.in1d(fore_inds, train_ind)).flatten()
valid_ind = np.argwhere(np.in1d(fore_inds, valid_ind)).flatten()
fore_train_ip = [ip[train_ind] for ip in [fore_demo, fore_times_ip, fore_values_ip, fore_varis_ip]]
fore_valid_ip = [ip[valid_ind] for ip in [fore_demo, fore_times_ip, fore_values_ip, fore_varis_ip]]
del fore_times_ip, fore_values_ip, fore_varis_ip, demo, fore_demo
fore_train_op = fore_op[train_ind]
fore_valid_op = fore_op[valid_ind]
del fore_op

print('lengths of rem_sub, fore_train_ip[1], fore_valid_ip[0]')
print(len(rem_sub), fore_train_ip[1].shape, fore_valid_ip[0].shape)
"""

In [ ]:

V=131
fore_max_len = 880
# Read data.
# data_path = '/home/mitarb/fracarolli/files/230613_STraTS_preprocessed/mimic_iii_preprocessed.pkl'
data, oc, train_ind, valid_ind, test_ind = pickle.load(open(data_path, 'rb'))

import itertools
if sepsis_check:
    test_ind = pd.read_csv('infec_test.csv', header=None)
    test_ind = list(itertools.chain(*test_ind.values.tolist()))
    test_ind = list(map(int, test_ind))
    print('hi')

means_stds = data.groupby("variable").agg({"mean":"first", "std":"first"})
#print(means_stds)
mean_std_dict = dict()
print(means_stds.keys)
for pos, row in means_stds.iterrows():
    print(pos)
    print(row)
    mean_std_dict[pos] = (float(row["mean"]), float(row["std"]))
print(mean_std_dict)
#raise Exception
# Filter labeled data in first 24h.
data = data.loc[data.ts_ind.isin(np.concatenate((train_ind, valid_ind, test_ind), axis=-1))]
data = data.loc[(data.hour >= 0) & (data.hour <= 48)]
old_oc = oc
oc = oc.loc[oc.ts_ind.isin(np.concatenate((train_ind, valid_ind, test_ind), axis=-1))]
# Fix age.
data.loc[(data.variable == 'Age') & (data.value > 200), 'value'] = 91.4
# Get y and N.
y = np.array(oc.sort_values(by='ts_ind')['in_hospital_mortality']).astype('float32')
print(y.shape)
#raise Exception
data['ts_ind'] = data['ts_ind'].astype(int) 
N = int(data.ts_ind.max() + 1)

# Create demographic/static data
# Get static data with mean fill and missingness indicator.
static_varis = ['Age', 'Gender']
ii = data.variable.isin(static_varis)
static_data = data.loc[ii]
data = data.loc[~ii]  # reduce data to non-demo/non-static
static_var_to_ind = inv_list(static_varis)
D = len(static_varis)
demo = np.zeros((int(N), D))
for row in tqdm(static_data.itertuples()):
    demo[int(row.ts_ind), static_var_to_ind[row.variable]] = row.value
# Normalize static data.
means = demo.mean(axis=0, keepdims=True)
stds = demo.std(axis=0, keepdims=True)
stds = (stds == 0)*1 + (stds != 0)*stds
demo = (demo-means)/stds

# Get N, V, var_to_ind.
varis = sorted(list(set(data.variable)))
var_to_ind = inv_list(varis, start=1)
data['vind'] = data.variable.map(var_to_ind)
data = data[['ts_ind', 'vind', 'hour', 'value']].sort_values(by=['ts_ind', 'vind', 'hour'])
# Add obs index.
data = data.sort_values(by=['ts_ind']).reset_index(drop=True)
data = data.reset_index().rename(columns={'index': 'obs_ind'})
data = data.merge(data.groupby('ts_ind').agg({'obs_ind': 'min'}).reset_index().rename(columns={
                                                            'obs_ind': 'first_obs_ind'}), on='ts_ind')
data['obs_ind'] = data['obs_ind'] - data['first_obs_ind']
# Find max_len.
data2 = data.loc[(data.hour >= 0) & (data.hour <= 24)]
max_len = data2.obs_ind.max()+1
print('max_len', max_len)
# Generate times_ip and values_ip matrices.
times_inp = np.zeros((N, max_len), dtype='float32')
values_inp = np.zeros((N, max_len), dtype='float32')
varis_inp = np.zeros((N, max_len), dtype='int32')
for row in tqdm(data2.itertuples()):
    ts_ind = row.ts_ind
    l = row.obs_ind
    times_inp[ts_ind, l] = row.hour
    values_inp[ts_ind, l] = row.value
    varis_inp[ts_ind, l] = row.vind

w=0

fore_in = []

pred_data = data.loc[(data.hour>=0)&(data.hour<=48)]
pred_data = pred_data.groupby(['ts_ind', 'vind']).agg({'value':'first'}).reset_index()
pred_data['vind_value'] = pred_data[['vind', 'value']].values.tolist()
pred_data = pred_data.groupby('ts_ind').agg({'vind_value':list}).reset_index()
pred_data['vind_value'] = pred_data['vind_value'].apply(f)   

obs_data = data.loc[(data.hour < 24) & (data.hour >= 0)]
resultdict = dict()
for ts_ind in obs_data.ts_ind:
    resultdict[ts_ind] = [[]]
obs_data = obs_data.loc[obs_data.ts_ind.isin(pred_data.ts_ind)]
obs_data = obs_data.groupby('ts_ind').head(fore_max_len)
obs_data = obs_data.groupby('ts_ind').agg({'vind': list, 'hour': list, 'value': list}).reset_index()

for pred_window  in range(0, 48, 1):
    print(pred_window)
    pred_data = data.loc[(data.hour >= w+pred_window) & (data.hour <= w+1 +pred_window)]
    pred_data = pred_data.groupby(['ts_ind', 'vind']).agg({'value': 'first'}).reset_index()
    pred_data['vind_value'+str(pred_window)] = pred_data[['vind', 'value']].values.tolist()
    pred_data = pred_data.groupby('ts_ind').agg({'vind_value'+str(pred_window): list}).reset_index()
    pred_data['vind_value'+str(pred_window)] = pred_data['vind_value'+str(pred_window)].apply(f)  # 721 entries with 2*129 vind_values
    obs_data = obs_data.merge(pred_data, on='ts_ind')


obs_data = obs_data.sort_values(by=['ts_ind']).reset_index(drop=True)
obs_data = obs_data.reset_index().rename(columns={'index': 'obs_ind'})
obs_data = obs_data.merge(data.groupby('ts_ind').agg({'obs_ind': 'min'}).reset_index().rename(columns={
                                                            'obs_ind': 'first_obs_ind'}), on='ts_ind')
obs_data['obs_ind'] = obs_data['obs_ind'] - obs_data['first_obs_ind']


N = int(obs_data.ts_ind.max() + 1)
times_inp = np.zeros((N, max_len), dtype='float32')
values_inp = np.zeros((N, max_len), dtype='float32')
varis_inp = np.zeros((N, max_len), dtype='int32')
for col in ['vind', 'hour', 'value']:
        obs_data[col] = obs_data[col].apply(pad)


times_inp=(np.array(list(obs_data.hour)))
values_inp=(np.array(list(obs_data.value)))
varis_inp=(np.array(list(obs_data.vind)))

print("times_shape_blub", times_inp.shape)
index=np.array([int(x) for x in list(obs_data.ts_ind)])
demo = demo[index]
blub = (np.array(list([list(obs_data['vind_value'+str(pred_window)]) for pred_window in range(0, 48, 1)])))
print("bleb", blub.shape)
op = np.swapaxes(blub, 0, 1)

weird_oc = oc.loc[oc.ts_ind.isin(obs_data.ts_ind)]
y = np.array(weird_oc.sort_values(by='ts_ind')['in_hospital_mortality']).astype('float32')
print(y.shape)
data.drop(columns=['obs_ind', 'first_obs_ind'], inplace=True)

train_ind = [x for x in train_ind if x < op.shape[0]]
valid_ind = [x for x in valid_ind if x < op.shape[0]]
test_ind = [x for x in test_ind if x < op.shape[0]]

train_input = op[train_ind]
valid_input = op[valid_ind]
test_input = op[test_ind]

# Generate 3 sets of inputs and outputs.
train_ip = [ip[train_ind] for ip in [demo, times_inp, values_inp, varis_inp]]
valid_ip = [ip[valid_ind] for ip in [demo, times_inp, values_inp, varis_inp]]
test_ip = [ip[test_ind] for ip in [demo, times_inp, values_inp, varis_inp]]
del demo, times_inp, values_inp, varis_inp  # warum wird demo nicht gelöscht?

if test_cond == 1:
    tr_ind = [divmod(tr, 12)[0] for tr in train_ind]
    va_ind = [divmod(tr, 12)[0] for tr in valid_ind]
    te_ind = [divmod(tr, 12)[0] for tr in test_ind]
    
    train_op = y[tr_ind]  # is a problem for the test case...
    valid_op = y[va_ind]
    test_op = y[te_ind]
else:
    train_op = y[train_ind]  # is a problem for the test case...
    valid_op = y[valid_ind]
    test_op = y[test_ind]
print('y is:', y)
del y
 

In [ ]:
def get_sofa(matrix, var_to_ind): #24x131 matrix
    # GCS: min_eye, min_motor, min_verbal = 5, 5, 5
    print(matrix.size())
    #raise Exception
    key ="GCS_eye"
    var_to_ind = {x:i-1 for x,i in var_to_ind.items()}
    a=matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]
    print(a)
    min_eye = min(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=4)
    key = "GCS_motor"
    min_motor = min(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=6)
    key = "GCS_verbal"
    min_verbal = min(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=5)
    

    GCS = min_eye + min_motor + min_verbal
    if GCS > 14: GCS_sofa = 0
    elif GCS > 12: GCS_sofa = 1
    elif GCS > 9:  GCS_sofa = 2
    elif GCS > 5:  GCS_sofa = 3
    else: GCS_sofa = 4
    #print('GCS_sofa is', GCS_sofa, ';     GCS is', GCS,'; GCS eye', min_eye, '; GCS motor', min_motor, '; GCS verbal', min_verbal)

    key = "Bilirubin (Total)"
    bilir = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
    if bilir > 12: bilir_sofa = 4
    elif bilir > 6: bilir_sofa = 3
    elif bilir > 2: bilir_sofa = 2
    elif bilir > 1.2: bilir_sofa = 1
    else: bilir_sofa = 0
    #print('bilir_Sofa is', bilir_sofa, ';   bilirubin is', bilir)
    
    # Coagulation (Platelets)
    key = "Platelet Count"
    plate = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=160)
    if plate > 150: plate_sofa = 0
    elif plate > 100: plate_sofa = 1
    elif plate > 50: plate_sofa = 2
    elif plate > 20: plate_sofa = 3
    else: plate_sofa = 4
    #print('plate_sofa is', plate_sofa, ';   platelet count is', plate)
    
    # print('Urinmenge 24h', sum(data_var[data_var['variable']=='Urine']['value2']))

    key = "Urine"
    urine = sum(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0])
    key = "Creatinine Blood"
    creat = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
    
    if (urine < 200) or (creat > 5): renal_sofa = 4
    elif  (urine < 500) or (creat > 3.5): renal_sofa = 3
    elif creat > 2.0: renal_sofa = 2
    elif creat > 1.2: renal_sofa = 1
    else: renal_sofa = 0
    #print('renal_sofa:',renal_sofa,';       urine 24:',urine,'; creat:', creat)
    
    CS_data = get_CS(matrix, var_to_ind)
    cs_sofa = CS_SOFA(CS_data)
    
    #cs_sofa = 0
    key="FiO2"
    fio2 = (matrix[:, var_to_ind[key]]*mean_std_dict[key][1]+mean_std_dict[key][0])
    key="PO2"
    po2 = (matrix[:, var_to_ind[key]]*mean_std_dict[key][1]+mean_std_dict[key][0])
    PaO2FiO2 = 100*po2/fio2
    print("size", PaO2FiO2.size())
    PaO2FiO2 = PaO2FiO2[torch.nonzero(PaO2FiO2, as_tuple=True)]
    pao2fio2 = min(PaO2FiO2)
    if pao2fio2<100: resp=4
    elif pao2fio2<200: resp=3
    elif pao2fio2<300:resp=2
    elif pao2fio2<400:resp=1
    else: resp=0
    return GCS_sofa, cs_sofa, resp, plate_sofa, bilir_sofa, renal_sofa

def get_CS(matrix, var_to_ind):
    #data_var = data_pat[data_pat['variable'].isin(['Dobutamine','Dopamine','Epinephrine','Norepinephrine','Weight'])]
    #data_var['value2'] = data_var['value']*data_var['std']+data_var['mean']
    
    #weight = min(data_var[data_var['variable']=='Weight']['value2'], default=80)  # set default weight to 80kg.
    key = "Weight"
    weight = min(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)], default=80)#*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
    key = "Dopamine"
    try:
        data_dop = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
        data_dop = data_dop /60/weight*1000
    except:
        data_dop = 0

    key = "Dobutamine"
    
    try:
        data_dobu = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
        data_dobu = data_dobu  /60/weight*1000
    except:
        data_dobu = 0
    key = "Epinephrine"
    
    try:
        data_epi = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
        data_epi = data_epi  /60/weight*1000
    except:
        data_epi = 0
    key = "Norepinephrine"
    
    try:
        data_nore = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
        data_nore = data_nore /60/weight*1000
    except:
        data_nore = 0
        


    key = "SBP"
    SBP = (matrix[:, var_to_ind[key]]*mean_std_dict[key][1]+mean_std_dict[key][0])

    key = "DBP"
    DBP = (matrix[:, var_to_ind[key]]*mean_std_dict[key][1]+mean_std_dict[key][0])

    MAP = 2/3 * DBP + 1/3 * SBP
    MAP = min(MAP[torch.nonzero(MAP, as_tuple=True)], default=100)
                 
        
    return MAP, data_dop, data_dobu, data_epi, data_nore 
    
def CS_SOFA(data):
    map = data[0]
    dop, dobu, epi, nore = data[1:5]
    # print('CS data: mdden', data)
    if (dop > 15) or (epi > 0.1) or (nore > 0.01): CS = 4
    elif (dop > 5) or (epi > 0) or (nore > 0): CS = 3
    elif (dop > 0) or (dobu > 0): CS = 2
    elif map < 70: CS = 1
    else: CS = 0
    # print('CS Sofa is:', CS)
    return CS 
    
factor_keys =["GCS_eye", "GCS_motor", "GCS_verbal", "Bilirubin (Total)", "Platelet Count", "Urine", "Creatinine Blood", "FiO2", "PO2", "Dopamine", "Dobutamine",
"Epinephrine", "Norepinephrine", "SBP", "DBP"]
factor_indices = sorted([var_to_ind[x]-1 for x in factor_keys])

In [ ]:

#Michi: die ganzen Argparses könnte man noch anders implementieren, wollte es nur schnell zum laufen bringen.
import argparse
parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

# basic config
parser.add_argument('--is_training', type=int, required=False, default=1, help='status')
parser.add_argument('--train_only', type=bool, required=False, default=False, help='perform training on full input dataset without validation and testing')
parser.add_argument('--model_id', type=str, required=False, default='test', help='model id')
parser.add_argument('--model', type=str, required=False, default='Autoformer',
                    help='model name, options: [Autoformer, Informer, Transformer]')

# data loader
parser.add_argument('--data', type=str, required=False, default='ETTm1', help='dataset type')
parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh1.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=24, help='input sequence length')
parser.add_argument('--label_len', type=int, default=0, help='start token length')
parser.add_argument('--pred_len', type=int, default=24, help='prediction sequence length')


# DLinear
parser.add_argument('--individual', action='store_true', default=False, help='DLinear: a linear layer for each variate(channel) individually')
# Formers 
#Michi: bisher nur default=3 zum laufen gebracht
parser.add_argument('--embed_type', type=int, default=3, help='0: default 1: value embedding + temporal embedding + positional embedding 2: value embedding + temporal embedding 3: value embedding + positional embedding 4: value embedding')
parser.add_argument('--enc_in', type=int, default=262, help='encoder input size') # DLinear with --individual, use this hyperparameter as the number of channels
parser.add_argument('--dec_in', type=int, default=131, help='decoder input size')
parser.add_argument('--c_out', type=int, default=131, help='output size')
parser.add_argument('--d_model', type=int, default=512, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=2, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=1, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=2048, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=1, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=2, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=10, help='train epochs')
parser.add_argument('--batch_size', type=int, default=32, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='mse', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')
parser.add_argument('--test_flop', action='store_true', default=False, help='See utils/tools for usage')

args = parser.parse_args(args=[])
import importlib


import matplotlib.pyplot as plt

In [ ]:
import models.InformerAutoregressiveWithForcedDecoding as autoformer
importlib.reload(autoformer)
model = autoformer.Model(args).cuda()


# Crossinteractions

In [ ]:
"""
torch.cuda.empty_cache()
import models.InformerAutoregressiveWithForcedDecoding as autoformer
importlib.reload(autoformer)
batch_size, lr, patience = 32, 0.0005, 6  # batch_size increased, patience 10 --> 6
d, N, he, dropout = 50, 2, 4, 0.2
model = autoformer.Model(args).cuda()


# Load pretrained weights here.
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#fore_savepath = 'mimic_iii_24h_strats_no_interp_with_ss_fore_blubdiblub.pytorch'
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=3)
model.load_state_dict(torch.load("mimic_iii_24h_strats_no_interp_with_ss_fore_informer_ims_fr_detach.pytorch"))
#model.load_state_dict(torch.load(fore_savepath))
model.eval()
y_true, y_pred = [], []
print(test_input)
pbar = tqdm(range(0, len(test_input), batch_size))
loss_list = []
test_loss = 0

keys = ["Dopamine", "Dobutamine", "Norepinephrine"]

#data_dop = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)

min_dict = dict()
max_dict = dict()
twentyfive_dict = dict()
median_dict = dict()
seventyfive_dict = dict()
matrix = torch.tensor(test_input)
for x in range(0, len(test_input)):

    for key in keys:
        try:
            min_data = min(matrix[x, :, var_to_ind[key]][torch.nonzero(matrix[x, :, var_to_ind[key]], as_tuple=True)])#*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
            if key not in min_dict:
                min_dict[key] = min_data
            else:
                min_dict[key] = min([min_data, min_dict[key]])
        except:
            pass
        try:
            max_data = max(matrix[x, :, var_to_ind[key]][torch.nonzero(matrix[x, :, var_to_ind[key]], as_tuple=True)])#*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)

            if key not in max_dict:
                max_dict[key] = max_data
            else:
                max_dict[key] = max([max_data, max_dict[key]])
        except:
            pass
        try:
            twentyfive_data = torch.quantile(matrix[x, :, var_to_ind[key]][torch.nonzero(matrix[x, :, var_to_ind[key]], as_tuple=True)], 0.25)#*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)

            if key not in twentyfive_dict:
                twentyfive_dict[key] = twentyfive_data.item()
            else:
                twentyfive_dict[key] = max([twentyfive_data, twentyfive_dict[key]])
        except:
            pass
        try:
            median_data = torch.quantile(matrix[x, :, var_to_ind[key]][torch.nonzero(matrix[x, :, var_to_ind[key]], as_tuple=True)], 0.50)#*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)

            if key not in median_dict:
                median_dict[key] = median_data.item()
            else:
                median_dict[key] = max([median_data, median_dict[key]])
        except:
            pass
        try:
            seventyfive_data = torch.quantile(matrix[x, :, var_to_ind[key]][torch.nonzero(matrix[x, :, var_to_ind[key]], as_tuple=True)], 0.75)#*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)

            if key not in seventyfive_dict:
                seventyfive_dict[key] = seventyfive_data.item()
            else:
                seventyfive_dict[key] = max([seventyfive_data, seventyfive_dict[key]])
        except:
            pass
print(min_dict, max_dict)
resultdict = dict()
for key in keys:
    maxes, mins, twentyfives, medians, seventyfives = [], [], [], [], []
    for start in pbar:
        print(start)
        matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
        input_matrix = matrix[:, :24]

        #torch.Size([32, 24, 129])
        input_mark = torch.arange(0, input_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        #torch.Size([32, 24])
        input_mask = input_matrix[:, :24, 131:]
        output_matrix = matrix[:, 24:, :131]
        #torch.Size([32, 24, 129])
        output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        #torch.Size([32, 24])
        output_mask = matrix[:, 24:, 131:]
        #torch.Size([32, 24, 129])
        #print(var_to_ind[key])
        #print(min_dict[key])
        #print(max_dict[key])
        with torch.no_grad():
            output_min = model(input_matrix, input_mark, None, None, trainn=False, change_dict = {var_to_ind[key]-1: min_dict[key]})
            output_max = model(input_matrix, input_mark, None, None, trainn=False, change_dict = {var_to_ind[key]-1: max_dict[key]})
            
            output_25 = model(input_matrix, input_mark, None, None, trainn=False, change_dict = {var_to_ind[key]-1: twentyfive_dict[key]})
            output_50 = model(input_matrix, input_mark, None, None, trainn=False, change_dict = {var_to_ind[key]-1: median_dict[key]})
            output_75 = model(input_matrix, input_mark, None, None, trainn=False, change_dict = {var_to_ind[key]-1: seventyfive_dict[key]})
        #print(output1, output2, output1==output2)
        for x in output_min:
            mins.append(x.cpu())
        for x in output_max:
            maxes.append(x.cpu())
        for x in output_25:
            twentyfives.append(x.cpu())
        for x in output_50:
            medians.append(x.cpu())
        for x in output_75:
            seventyfives.append(x.cpu())
    print(len(maxes), maxes[0].size())
    resultdict[(key, "min")]=mins
    resultdict[(key, "max")]=maxes
    resultdict[(key, "25")]=twentyfives
    resultdict[(key, "50")]=medians
    resultdict[(key, "75")]=seventyfives
import pickle

with open('crossinteractions.pickle', 'wb') as handle:
    pickle.dump([resultdict, var_to_ind, min_dict, max_dict, mean_std_dict], handle)
"""

# 8 Sofa Analysis IMS Informer TF

# 7 Sofa Analysis DMS Informer

# 10 Sofa Analysis IMS Informer Backprop

# 9 Sofa Analysis IMS Informer Detach

In [ ]:

import models.InformerAutoregressiveFull as autoformer
importlib.reload(autoformer)
batch_size, lr, patience = 32, 0.0005, 6  # batch_size increased, patience 10 --> 6
d, N, he, dropout = 50, 2, 4, 0.2
model = autoformer.Model(args).cuda()
# Load pretrained weights here.
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#fore_savepath = 'mimic_iii_24h_strats_no_interp_with_ss_fore_blubdiblub.pytorch'
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=3)
model.load_state_dict(torch.load("mimic_iii_24h_strats_no_interp_with_ss_fore_informer_ims_fr_detach2_full.pytorch"))
#model.load_state_dict(torch.load(fore_savepath))
model.eval()
y_true, y_pred = [], []
print(test_input)
pbar = tqdm(range(0, len(test_input), batch_size))
loss_list = []
test_loss = 0
test_loss_part1 = 0
test_loss_part2 = 0
test_loss_sofa_variables = 0
sofa_losses = []
accuracy_list = []
TP, TN, FP, FN = 0, 0, 0, 0
test_loss_sum_square_total = 0
test_loss_part1_sum_square_total = 0
test_loss_part2_sum_square_total = 0
test_loss_sofa_sum_square_total = 0
for start in pbar:
    print(start)
    matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
    input_matrix = matrix[:, :24]

    #torch.Size([32, 24, 129])
    input_mark = torch.arange(0, input_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    input_mask = input_matrix[:, :24, 131:]
    output_matrix = matrix[:, 24:, :131]
    #torch.Size([32, 24, 129])
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    output_mask = matrix[:, 24:, 131:]
    
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    dec_inp = torch.zeros_like(output_matrix[:, -args.pred_len:, :]).float()
    #torch.Size([32, 24, 129])
    with torch.no_grad():
        output = model(input_matrix, input_mark, dec_inp, output_mark, trainn=False)
        loss = output_mask[:, -args.pred_len:, :]*(
        output-output_matrix[:, -args.pred_len:, :])**2
    loss_list.extend(loss.sum(axis=-1).mean(axis=-1).detach().cpu().tolist())
    for outpu, real, mask, input_mat in zip(output, output_matrix, output_mask, input_matrix):
        a = (get_sofa(outpu*mask, var_to_ind))
        b = (get_sofa(real, var_to_ind))
        c = (get_sofa(input_mat, var_to_ind)) #SOFA Source (First Day)
        prediction_sofa_difference = (sum(a)-sum(c)) >= 2
        gold_sofa_difference = (sum(b)-sum(c)) >= 2
        accuracy_list.append(prediction_sofa_difference == gold_sofa_difference)
        if gold_sofa_difference:
            if prediction_sofa_difference: TP +=1
            else: FP += 1
        else:
            if prediction_sofa_difference: TN +=1
            else: FN += 1
        c, d = torch.tensor(a, dtype=torch.float32), torch.tensor(b, dtype=torch.float32)
        sofa_losses.append(((c-d)**2).sum(axis=-1).mean().item())
    loss_old = loss
    loss = loss.sum(axis=-1).mean(axis=-1).sum()
    loss_part1 = loss_old[:, :8].sum(axis=-1).mean()
    loss_part2 = loss_old[:, 8:].sum(axis=-1).mean()
    test_loss+=loss
    test_loss_part1+=loss_part1
    test_loss_part2+=loss_part2
    test_loss_sofa_variables+=loss_old[:, :, factor_indices].sum(axis=-1).mean()
    test_loss_sum_square_total += torch.sum(loss_old.sum(axis=-1).mean(axis=-1)**2)
    test_loss_part1_sum_square_total += torch.sum(loss_old[:, :8].sum(axis=-1).mean(axis=-1)**2)
    test_loss_part2_sum_square_total += torch.sum(loss_old[:, 8:].sum(axis=-1).mean(axis=-1)**2)
    test_loss_sofa_sum_square_total += torch.sum(loss_old[:, :, factor_indices].sum(axis=-1).mean(axis=-1)**2)
test_loss_p = test_loss/len(test_input)
se_loss_p = torch.sqrt(test_loss_sum_square_total / len(test_input) - test_loss_p**2)/len(test_input)
test_loss_part1_p = test_loss_part1*batch_size/len(test_input)
test_loss_part2_p = test_loss_part2*batch_size/len(test_input)
test_loss_sofa_variables_p = test_loss_sofa_variables*batch_size/len(test_input)
sofa_loss = sum(sofa_losses)/len(sofa_losses)
accuracy_sepsis = sum(accuracy_list)/len(accuracy_list)
se_accuracy_sepsis = np.sqrt(accuracy_sepsis * (1-accuracy_sepsis)/len(test_input))
se_loss_part1_p = torch.sqrt(test_loss_part1_sum_square_total / len(test_input) - test_loss_part1_p**2)/len(test_input)
se_loss_part2_p = torch.sqrt(test_loss_part2_sum_square_total / len(test_input) - test_loss_part2_p**2)/len(test_input)
se_loss_sofa_p  = torch.sqrt(test_loss_sofa_sum_square_total / len(test_input)  - test_loss_sofa_variables_p**2)/len(test_input)
print(test_loss_p, test_loss_part1_p, test_loss_part2_p, sofa_loss, test_loss_sofa_variables_p, accuracy_sepsis)
print('Informer_IMS_SF_D',test_loss_p.item(), test_loss_part1_p.item(), test_loss_part2_p.item(), sofa_loss, test_loss_sofa_variables_p.item(), accuracy_sepsis, sep=',',file=open('results.txt','a'))



model_name = '9&Dense&IMS&SF&No&'
MSE_string = "{:.3f}".format(test_loss_p.item()) + "$_{[" + "{:.3f}".format(test_loss_p.item()-1.96*se_loss_p.item()) +','+ "{:.3f}".format(test_loss_p.item()+1.96*se_loss_p.item()) + ']}$&'
SOFA_string = "{:.3f}".format(sofa_loss)+ '$_{['+"{:.3f}".format(sofa_loss-1.96*se_loss_sofa_p.item())+','+"{:.3f}".format(sofa_loss+1.96*se_loss_sofa_p.item())+ ']}$&'
MSE8_string = "{:.3f}".format(test_loss_part1_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part1_p.item()-1.96*se_loss_part1_p.item()) +','+ "{:.3f}".format(test_loss_part1_p.item()+1.96*se_loss_part1_p.item()) + ']}$&'
MSE16_string = "{:.3f}".format(test_loss_part2_p.item()) + "$_{[" + "{:.3f}".format(test_loss_part2_p.item()-1.96*se_loss_part2_p.item()) +','+ "{:.3f}".format(test_loss_part2_p.item()+1.96*se_loss_part2_p.item()) + ']}$&'
SOFA_acc_string = "{:.2f}".format(accuracy_sepsis*100)+ '$_{['+"{:.2f}".format(100*(accuracy_sepsis-1.96*se_accuracy_sepsis))+','+"{:.2f}".format(100*(accuracy_sepsis+1.96*se_accuracy_sepsis))+ ']}$&'
print(accuracy_sepsis)
print(MSE_string)
print(SOFA_string)

# Autof SOFA Analysis Autoformer

In [ ]:

import models.InformerAutoregressiveFullRegression as autoformer
importlib.reload(autoformer)
batch_size, lr, patience = 32, 0.0005, 6  # batch_size increased, patience 10 --> 6
d, N, he, dropout = 50, 2, 4, 0.2
model = autoformer.Model(args).cuda()
# Load pretrained weights here.
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#fore_savepath = 'mimic_iii_24h_strats_no_interp_with_ss_fore_blubdiblub.pytorch'
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=3)
model.load_state_dict(torch.load("informer_regression.pytorch"))
#model.load_state_dict(torch.load(fore_savepath))
model.eval()
y_true, y_pred = [], []
print(test_input)
pbar = tqdm(range(0, len(test_input), batch_size))
loss_list = []
test_loss = 0
test_loss_part1 = 0
test_loss_part2 = 0
test_loss_sofa_variables = 0
sofa_losses = []
accuracy_list = []
TP, TN, FP, FN = 0, 0, 0, 0
test_loss_sum_square_total = 0
test_loss_part1_sum_square_total = 0
test_loss_part2_sum_square_total = 0
test_loss_sofa_sum_square_total = 0
for start in pbar:
    print(start)
    matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
    input_matrix = matrix[:, :24]

    #torch.Size([32, 24, 129])
    input_mark = torch.arange(0, input_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    input_mask = input_matrix[:, :24, 131:]
    output_matrix = matrix[:, 24:, :131]
    #torch.Size([32, 24, 129])
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    output_mask = matrix[:, 24:, 131:]
    
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    dec_inp = torch.zeros_like(output_matrix[:, -args.pred_len:, :]).float()
    #torch.Size([32, 24, 129])
    with torch.no_grad():
        output = model(input_matrix, input_mark, dec_inp, output_mark, trainn=False)
    loss = 0
    for outpu, real, mask, input_mat in zip(output, output_matrix, output_mask, input_matrix):
        a = outpu.item()
        b = (get_sofa(real, var_to_ind))
        c = (get_sofa(input_mat, var_to_ind)) #SOFA Source (First Day)
        
        prediction_sofa_difference = (a-sum(c)) >= 2
        gold_sofa_difference = (sum(b)-sum(c)) >= 2
        print(a, sum(b))
        loss_term = (outpu.squeeze()-sum(b))**2
        loss+=loss_term
        accuracy_list.append(prediction_sofa_difference == gold_sofa_difference)
        if gold_sofa_difference:
            if prediction_sofa_difference: TP +=1
            else: FN += 1
        else:
            if prediction_sofa_difference: FP +=1
            else: TN += 1
    loss_old = loss
    loss = loss.sum(axis=-1).mean(axis=-1).sum()

    test_loss+=loss

print(TP, FP, FN, TN)
print()
accuracy_sepsis = sum(accuracy_list)/len(accuracy_list)
se_accuracy_sepsis = np.sqrt(accuracy_sepsis * (1-accuracy_sepsis)/len(test_input))
print(accuracy_sepsis)

In [ ]:

import models.InformerAutoregressiveFullRegression as autoformer
importlib.reload(autoformer)
batch_size, lr, patience = 32, 0.0005, 6  # batch_size increased, patience 10 --> 6
d, N, he, dropout = 50, 2, 4, 0.2
model = autoformer.Model(args).cuda()
# Load pretrained weights here.
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#fore_savepath = 'mimic_iii_24h_strats_no_interp_with_ss_fore_blubdiblub.pytorch'
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=3)
model.load_state_dict(torch.load("informer_regression2.pytorch"))
#model.load_state_dict(torch.load(fore_savepath))
model.eval()
y_true, y_pred = [], []
print(test_input)
pbar = tqdm(range(0, len(test_input), batch_size))
loss_list = []
test_loss = 0
test_loss_part1 = 0
test_loss_part2 = 0
test_loss_sofa_variables = 0
sofa_losses = []
accuracy_list = []
TP, TN, FP, FN = 0, 0, 0, 0
test_loss_sum_square_total = 0
test_loss_part1_sum_square_total = 0
test_loss_part2_sum_square_total = 0
test_loss_sofa_sum_square_total = 0
for start in pbar:
    print(start)
    matrix = torch.tensor(test_input[start:start+batch_size], dtype=torch.float32).cuda()
    input_matrix = matrix[:, :24]

    #torch.Size([32, 24, 129])
    input_mark = torch.arange(0, input_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    input_mask = input_matrix[:, :24, 131:]
    output_matrix = matrix[:, 24:, :131]
    #torch.Size([32, 24, 129])
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    #torch.Size([32, 24])
    output_mask = matrix[:, 24:, 131:]
    
    output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
    dec_inp = torch.zeros_like(output_matrix[:, -args.pred_len:, :]).float()
    #torch.Size([32, 24, 129])
    with torch.no_grad():
        output = model(input_matrix, input_mark, dec_inp, output_mark, trainn=False)
    loss = 0
    for outpu, real, mask, input_mat in zip(output, output_matrix, output_mask, input_matrix):
        a = outpu.item()
        b = (get_sofa(real, var_to_ind))
        c = (get_sofa(input_mat, var_to_ind)) #SOFA Source (First Day)
        
        prediction_sofa_difference = (a-sum(c)) >= 2
        gold_sofa_difference = (sum(b)-sum(c)) >= 2
        print(a, sum(b))
        loss_term = (outpu.squeeze()-sum(b))**2
        loss+=loss_term
        accuracy_list.append(prediction_sofa_difference == gold_sofa_difference)
        if gold_sofa_difference:
            if prediction_sofa_difference: TP +=1
            else: FN += 1
        else:
            if prediction_sofa_difference: FP +=1
            else: TN += 1
    loss_old = loss
    loss = loss.sum(axis=-1).mean(axis=-1).sum()

    test_loss+=loss

print(TP, FP, FN, TN)
print()
accuracy_sepsis = sum(accuracy_list)/len(accuracy_list)
se_accuracy_sepsis = np.sqrt(accuracy_sepsis * (1-accuracy_sepsis)/len(test_input))
print(accuracy_sepsis)